In [ ]:
!pip install tensorflow keras-tuner opencv-python matplotlib

In [ ]:
import tensorflow as tf
import os
import pickle

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! mkdir ~/.kaggle
!cp /content/drive/MyDrive/Model_Training/kaggle_API_Authorisation_creds/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d iamsouravbanerjee/indian-food-images-dataset

In [ ]:
! unzip indian-food-images-dataset.zip -d indian-food-images-dataset

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
DATA_PATH = 'indian-food-images-dataset/Indian Food Images/Indian Food Images'
BATCH_SIZE = 32
IMG_HEIGHT, IMG_WIDTH = 224, 224
IMG_SIZE = (IMG_HEIGHT, IMG_WIDTH, 3)

train_data = tf.keras.utils.image_dataset_from_directory(
    DATA_PATH,
    label_mode='categorical',
    color_mode='rgb',
    validation_split=0.2,
    subset="training",
    seed=42,
    shuffle=True,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
    )

val_data = tf.keras.utils.image_dataset_from_directory(
    DATA_PATH,
    label_mode='categorical',
    color_mode='rgb',
    validation_split=0.2,
    subset="validation",
    seed=42,
    shuffle=False,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
    )

class_names = train_data.class_names

# optimize
train_data = train_data.prefetch(tf.data.AUTOTUNE)
val_data = val_data.prefetch(tf.data.AUTOTUNE)

In [ ]:
from tensorflow import keras
from keras.applications import ResNet50V2, VGG19, InceptionV3

base_model = ResNet50V2(
    include_top=False, # Exclude ImageNet classifier at the top
    weights='imagenet',
    input_shape=IMG_SIZE
    )

In [ ]:
# Freeze the base_model
base_model.trainable = False

In [ ]:
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomTranslation, RandomContrast

data_augmentation = keras.Sequential(
    [RandomFlip('horizontal'), 
     RandomRotation(factor=(-0.025, 0.025)),
     RandomTranslation(height_factor=0.1, width_factor=0.1),
     RandomContrast(factor=0.1)
     ])

In [ ]:
# Create new model on top
inputs = keras.Input(shape=IMG_SIZE)
x = data_augmentation(inputs)

x = keras.applications.resnet_v2.preprocess_input(x)
x = base_model(x, training=False)

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D, BatchNormalization, Dropout, Flatten, Dense

# Rebuild top layers
x = GlobalAveragePooling2D()(x)
x = BatchNormalization()(x)
x = Dropout(0.2)(x)  # Regularize with dropout

outputs = Dense(len(class_names), activation="softmax", name="pred")(x)

cooksmart_model = keras.Model(inputs, outputs)

In [ ]:
cooksmart_model.summary()

In [ ]:
from tensorflow.keras.metrics import *
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

metrics = [CategoricalAccuracy(), Precision(), Recall()]

cooksmart_model.compile(
    loss = CategoricalCrossentropy(),
    optimizer = Adam(learning_rate=0.001),
    metrics = metrics
    )

es = EarlyStopping(patience=6, monitor='val_loss', mode='min', restore_best_weights=True)

In [ ]:
cooksmart_history = cooksmart_model.fit(
      train_data, 
      epochs=25, 
      validation_data=val_data,
      callbacks=[es],
      verbose=1
      )

In [ ]:
cooksmart_model.save('resnet-model-23-02-21.h5')

In [ ]:
pickle.dump(cooksmart_model,open('cooksmartmodel.pkl','wb'))

In [ ]:
pickle_model = pickle.load(open('cooksmartmodel.pkl','rb'))

In [ ]:
FOOD_CLASSES = {x:y for x,y in enumerate(class_names)}
MODEL_DIR = '/content/resnet-model-23-02-21.h5'
MODEL = keras.models.load_model(MODEL_DIR)

def predict_image_class(img_path:str) -> str:
  img = keras.preprocessing.image.load_img(img_path, target_size=IMG_SIZE)
  img_array = keras.preprocessing.image.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0)  # Create batch axis

  predictions = MODEL.predict(img_array)

  return FOOD_CLASSES[predictions.argmax(axis=1)[0]]

In [ ]:
TEST_IMG_DIR = "/content/manasi/abc.jfif"
print(f"Prediction - {predict_image_class(TEST_IMG_DIR)}")

In [ ]:
from google.colab import files
files.download('/content/resnet-model-23-02-21.h5')